In [ ]:
!pip install datasets llama-index==0.10.34 langchain-openai==0.1.6 "nemoguardrails[openai]==0.8.0" openai==1.25.1 chromadb==0.5.0 wandb==0.16.6 -qqq
!pip install llama-index-callbacks-wandb==0.1.2 -qqq

# **데이터셋 다운로드 및 API 키 설정**

In [ ]:
import os
from datasets import load_dataset

# 벡터 검색 기반만 사용할거면 필요하지 않음
os.environ["OPENAI_API_KEY"] = "자신의 OpenAI API 키 입력"

dataset = load_dataset('klue', 'mrc', split='train')
dataset[0]

# **실습 데이터 중 첫 100개를 뽑아 임베딩 벡터로 변환하고 저장**

In [ ]:
from llama_index.core import Document, VectorStoreIndex

text_list = dataset[:100]['context']
# Document -> 라마인덱스에서 사용하는 Document 객체 생성
documents = [Document(text=t) for t in text_list]

# 인덱스 만들기

# VectorStoreIndex.from_documents() → 문서들을 벡터로 변환하여 검색할 수 있는 인덱스를 생성
# 내부적으로 텍스트를 임베딩(숫자 벡터)으로 변환하고 검색 가능한 형태로 저장
index = VectorStoreIndex.from_documents(documents)

# **100개의 기사 본문 데이터에서 질문과 가까운 기사 찾기**

In [ ]:
print(dataset[0]['question']) # 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?

# index.as_retriever() → LlamaIndex가 제공하는 검색 엔진을 생성
# verbose=True → 검색 과정의 상세 정보를 출력
retrieval_engine = index.as_retriever(similarity_top_k=5, verbose=True)

# retrieval_engine.retrieve() -> 검색 실행
response = retrieval_engine.retrieve(
    dataset[0]['question']
)

print(len(response)) # 출력 결과: 5

# 검색결과 0번째 인덱스 출력
print(response[0].node.text)

# **라마인덱스를 활용해 검색 증강 생성 수행하기**

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=1)

response = query_engine.query(
    dataset[0]['question']
)

print(response)
# 장마전선에서 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 한 달 정도입니다.

# **라마인덱스 내부에서 검색 증강 생성을 수행하는 과정**

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# 검색을 위한 Retriever 생성
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=1,
)

# 검색 결과를 질문과 결합하는 synthesizer
response_synthesizer = get_response_synthesizer()

# 위의 두 요소를 결합해 쿼리 엔진 생성
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostporcessor(similarity_cutoff==0.7)],
)

# RAG 수행
response = query_engine.query("북태평양 기단과 오호츠크해 기단이 만나 국내에 머무리는 기간은?")
print(response)
# 장마전선에서 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 한 달 가량입니다.